In [ ]:
import torch
from torch.autograd import Variable
from utee import selector
model_raw, ds_fetcher, is_imagenet = selector.select('resnet50')
ds_val = ds_fetcher(batch_size=10, train=False, val=True)
for idx, (data, target) in enumerate(ds_val):
    data =  Variable(torch.FloatTensor(data)).cuda()
    output = model_raw(data)

In [1]:
import os
import sys
os.chdir("/data/adv_geom/drg/")
from models.classifier import Classifier

In [2]:
! ls ../

datasets  drg  dumps  notebooks


In [3]:
from training.train_classifier import TrainClassifier

[1802210643] [Warning] running without display


In [6]:
%tb
args = {
    "train_images_file": "/data/adv_geom/datasets/emnist/train_images.h5",
    "train_codes_file": "/data/adv_geom/datasets/emnist/train_labels.h5",
    "test_images_file": "/data/adv_geom/datasets/emnist/test_images.h5",
    "test_codes_file": "/data/adv_geom/datasets/emnist/test_labels.h5",
    "state_file": "/data/adv_geom/dumps/emnist/classifier.pth.tar",
    "label_index": 0,
    "lr_decay": 0.95,
    "weight_decay": 0.0001,
    "training_samples": 160000,
    "validation_samples": 40000,
    "random_samples": True,
    "network_architecture": "mlp",
    "gpu_id": 1
}

sys.argv = ["-" + i + "=" + str(args[i]) if type(args[i]) != type(True) else "-" + i for i in args]


program = TrainClassifier(args=sys.argv)

SystemExit: 2

usage: train_images.h5 [-h] [-train_images_file TRAIN_IMAGES_FILE]
                       [-train_codes_file TRAIN_CODES_FILE]
                       [-test_images_file TEST_IMAGES_FILE]
                       [-test_codes_file TEST_CODES_FILE]
                       [-state_file STATE_FILE] [-log_file LOG_FILE]
                       [-training_file TRAINING_FILE]
                       [-testing_file TESTING_FILE] [-loss_file LOSS_FILE]
                       [-error_file ERROR_FILE] [-gradient_file GRADIENT_FILE]
                       [-label_index LABEL_INDEX]
                       [-training_samples TRAINING_SAMPLES]
                       [-validation_samples VALIDATION_SAMPLES]
                       [-test_samples TEST_SAMPLES] [-early_stopping]
                       [-random_samples] [-batch_size BATCH_SIZE]
                       [-epochs EPOCHS] [-weight_decay WEIGHT_DECAY]
                       [-logit_decay LOGIT_DECAY] [-no_gpu] [-skip SKIP]
                       [-l

SystemExit: 2

In [9]:
program.load_data()

[1802210451] [Training] read /data/adv_geom/datasets/emnist/train_images.h5
[1802210451] [Training] read /data/adv_geom/datasets/emnist/test_images.h5
[1802210451] [Training] resolution 28
[1802210451] 240000
[1802210451] [Training] read /data/adv_geom/datasets/emnist/train_labels.h5
[1802210451] [Training] read /data/adv_geom/datasets/emnist/test_labels.h5
[1802210451] [Training] using 160000 training samples
[1802210451] [Training] found 10 classes
[1802210451] [Training] dataset not balanced, class 0 should have 16000 samples but has 16062
[1802210451] [Training] dataset not balanced, class 1 should have 16000 samples but has 15902
[1802210451] [Training] dataset not balanced, class 2 should have 16000 samples but has 15827
[1802210451] [Training] dataset not balanced, class 3 should have 16000 samples but has 16098
[1802210451] [Training] dataset not balanced, class 4 should have 16000 samples but has 16016
[1802210451] [Training] dataset not balanced, class 5 should have 16000 sam

In [10]:
program.load_model_and_scheduler()

[1802210451] [Training] using 1 input channels
[1802210451] [Training] loaded /data/adv_geom/dumps/emnist/classifier.pth.tar
[1802210451] [Training] model is not CUDA
[1802210451] [Training] loaded model
[1802210451] [Training] read /data/adv_geom/dumps/emnist/training.h5
[1802210451] [Training] read /data/adv_geom/dumps/emnist/testing.h5
[1802210451] (view0, View)
(lin1, Linear)
(act1, ReLU)
(bn1, BatchNorm1d)
(lin2, Linear)
(act2, ReLU)
(bn2, BatchNorm1d)
(lin3, Linear)
(act3, ReLU)
(bn3, BatchNorm1d)
(lin4, Linear)
(act4, ReLU)
(bn4, BatchNorm1d)
(logits, Linear)



In [25]:
test_image = program.test_images[0:8]
test_codes = program.test_codes[0:8]

In [26]:
import torch

with torch.no_grad():
    program.model.cuda()
    program.model.eval()
    test_image = torch.Tensor(test_image).cuda()
    logits = program.model(test_image)
    

In [31]:
torch.max(logits, axis=1)[1]

tensor([0, 9, 7, 9, 2, 2, 3, 1], device='cuda:1')

In [32]:
test_codes

array([0, 9, 7, 9, 2, 2, 3, 1])

In [33]:
program.model.cpu()

Classifier(
  (view0): View()
  (lin1): Linear(in_features=784, out_features=1024, bias=True)
  (act1): ReLU(inplace=True)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin2): Linear(in_features=1024, out_features=1024, bias=True)
  (act2): ReLU(inplace=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin3): Linear(in_features=1024, out_features=1024, bias=True)
  (act3): ReLU(inplace=True)
  (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin4): Linear(in_features=1024, out_features=1024, bias=True)
  (act4): ReLU(inplace=True)
  (bn4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (logits): Linear(in_features=1024, out_features=10, bias=True)
)

In [36]:
test_image.cpu()

tensor([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         ...,

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],


        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
